# 設定

In [ ]:
channel = "ibm_quantum" # ibm_cloud, ibm_quantum
backend_name = "ibm_osaka"
token="" # あなたのトークンを入力してください

# ライブラリのインポート

In [ ]:
from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# 共通ロジック

In [ ]:
service = QiskitRuntimeService(channel=channel, token=token)
backend = service.backend(backend_name)
sampler = Sampler(backend)

def run_remote_job(qc):
    qc_transpiled = transpile(qc,backend)
    job = sampler.run([qc_transpiled])
    result = job.result()
    pub_result = result[0]
    return pub_result

# テスト

## ケース01：直列負荷

In [ ]:
# 回路作成
qc_01 = QuantumCircuit(1,1)
qc_01.h(0)
for i in range(128):
    qc_01.z(0)
qc_01.measure_all()

In [ ]:
# リモートジョブ実行
pub_result_01 = run_remote_job(qc_01)
counts_01 = pub_result_01.data.meas.get_counts()
plot_histogram(counts_01)

## ケース02：並列負荷(独立)

In [ ]:
# 回路作成
qc_02 = QuantumCircuit(16,1)
for i in range(16):
    qc_02.h(0)
qc_02.measure_all()

In [ ]:
# リモートジョブ実行
pub_result_02 = run_remote_job(qc_02)
counts_02 = pub_result_02.data.meas.get_counts()
print(counts_02)

## ケース03：並列負荷(エンタングル)

In [ ]:
# 回路作成
qc_03 = QuantumCircuit(16,1)
qc_03.h(0)
for i in range(1,16):
    qc_03.cx(0,i)
qc_03.measure_all()

In [ ]:
# リモートジョブ実行
pub_result_03 = run_remote_job(qc_03)
counts_03 = pub_result_03.data.meas.get_counts()
for item in counts_03.items():
    print(item)